In [2]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt


In [3]:
def loaddata(i):
    path = "input{}.stock".format(i)
    f = open(path, "r")
    txt = f.readlines()
    f.close()
    stockLength = int(txt[0][14:-1])
    n = len(txt[3].split(', '))
    req = list(map(int,txt[3].split(', ')))
    ans = txt[-1][8:-1]
    return stockLength, n, req, ans

In [4]:
class Order:
    def __init__(self, id, length):
        self.id = id
        self.len = length        

In [5]:
class Problem:
    def __init__(self, T, coolingRate, stockLength, markovLength, 
                 orders, stopCriterion):
        self.T = T
        self.coolingRate = coolingRate
        self.numberOfOrder = len(orders)
        self.stockLength = stockLength
        self.markovLength = markovLength
        self.orders = orders
        self.bestAns = math.inf
        self.bestArrangement = []
        self.stopCriterion = stopCriterion
        self.solution = []
        
    def train(self):
        self.initializer()
        outerCounter = 0
        itter = 0
        while outerCounter < self.stopCriterion:
            if itter%150 == 0: print(str(itter)+': '+str(outerCounter)+' '+str(self.fittness(self.solution)))
            if itter%1000 == 0:
              for i in self.solution:
                print(i.len,end=',')
     
            itter += 1
            self.T = self.T * self.coolingRate + 0.0001
            preSolution = self.solution[:]
            self.innerLoop()
            if preSolution == self.solution:
                outerCounter += 1
        return self.bestAns, self.bestArrangement
        
    def initializer(self):
        for i,length in enumerate(self.orders):
            order = Order(i,length)
            self.solution.append(order)
        
        random.shuffle(self.solution)
    def innerLoop(self):
        n = 0
        while n <= self.markovLength: 
            nextsolution = self.swap(self.solution)
            self.checkNeighborhood(nextsolution)
            n += 1
            
        
    def checkNeighborhood(self,nextsolution):    
        nextcost = self.fittness(nextsolution)
        delta = nextcost - self.fittness(self.solution)
        if delta < 0:    
            self.solution = nextsolution
        else:
            maxProb = np.exp((-delta)/self.T)
            prob = random.random()
            if prob < maxProb:
                self.solution = nextsolution
                
        if self.bestAns > nextcost:
            self.bestAns = nextcost
            self.bestArrangement = nextsolution
        
    
    def fittness(self, solution):
        rolls = 1
        remain = self.stockLength
        for order in solution:
            if order.len <= remain:
                remain -= order.len
            else:
                rolls += 1
                remain = self.stockLength - order.len
                
        return rolls
        
    
    def swap(self, arr):
        newarr = arr[:]
        point1 = random.randint(0, self.numberOfOrder-1)
        point2 = random.randint(0, self.numberOfOrder-1)
        newarr[point1], newarr[point2] = newarr[point2], newarr[point1]
        return newarr

    def printResult(self):
        remain = self.stockLength
        for order in self.solution:
            if order.len < remain:
                remain -= order.len
                print(order.id, end=' ') 
            else:
                print('------'+str(remain))
                print(order.id, end=' ')
                remain = self.stockLength - order.len
                
        

In [6]:
stockLength, n, req, ans = loaddata(1)
print((len(req)))
solve = Problem(T = 1
            , coolingRate = 0.9
            , stockLength = stockLength
            , markovLength = 3
            , orders = req
            , stopCriterion = 1000)
rolls, ans = solve.train()
print(rolls)
for i in ans:
    print(i.len,end=' ')

140
0: 0 65
125,171,106,409,149,149,18,868,23,618,412,71,46,116,716,187,230,84,555,306,609,592,424,218,186,501,278,246,517,86,211,549,286,106,80,264,69,149,295,224,88,118,92,506,284,581,988,251,788,518,370,126,266,79,368,315,119,371,460,266,53,365,627,914,117,292,515,437,248,106,107,88,463,145,337,678,241,170,99,144,312,753,660,43,805,686,653,662,356,987,402,280,441,648,109,557,525,135,283,115,286,495,33,125,544,351,181,632,249,232,75,70,78,557,672,405,967,354,788,457,346,333,60,148,532,689,753,933,268,507,123,92,106,557,312,45,414,61,301,180,150: 15 54
300: 38 54
450: 65 53
600: 96 53
750: 149 52
900: 211 52
115,662,716,653,409,549,525,283,180,70,312,286,149,125,23,78,135,80,618,86,266,232,402,84,241,354,315,788,88,581,301,988,230,18,689,92,460,405,79,118,224,518,33,412,149,126,148,106,557,144,106,346,627,145,53,278,211,280,119,788,106,660,60,116,125,532,264,46,437,506,592,371,967,171,181,424,648,306,333,632,249,351,356,43,292,672,284,544,88,933,106,286,45,557,69,517,218,109,686,515,4